# This is a mostly non-technical demo notebook for a CNN model that predicts March Madness games
##### Below are imports, more than are in use, just extra tools in case.

In [110]:
from __future__ import print_function
import sys
import csv
import tensorflow as tf
import sklearn
import en_core_web_sm
import keras
from keras import layers
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras_tqdm import TQDMNotebookCallback
from keras.models import Sequential
from keras.layers import Dropout, Activation
from keras.optimizers import SGD
from IPython import display
from keras.preprocessing.sequence import pad_sequences
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#pd.set_option('display.max_columns', None)
pd.options.display.max_columns = 10
pd.set_option('display.max_rows', None)
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'monospace'
from matplotlib.ticker import MaxNLocator
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import os, eli5, shap
np.set_printoptions(threshold=sys.maxsize)
from eli5.sklearn import PermutationImportance
from pdpbox import pdp

In [112]:
#Create dataframes that will be used for performance comparison later
pastModels=[[2017,1,.43857],[2017,2,.44981],[2017,3,.45373],[2017,4,.46100],[2017,5,.46107],[2018,1,.53194],[2018,2,.53693],[2018,3,.54013],[2018,4,.54967],[2018,5,.54987],[2019,1,.41477],[2019,2,.42012],[2019,3,.42698],[2019,4,.42788],[2019,5,.43148]]
cols=['Year','Rank','LogLoss']
pastScores=pd.DataFrame(pastModels, columns=cols)
pastMadness=[[2017,65.7,82,145,99.99],[2018,57,81,128,98.8],[2019,74,124,163,99.99]]
cols=['Year','avg_score','higherSeed_score','myScore','percentile_myScore']
pastMadness=pd.DataFrame(pastMadness, columns=cols)

## Importing the data here and slicing it in different ways for training and testing model
##### We will only be showing testing model performance

In [127]:
tourney_data=pd.read_csv("tourney_game_master.csv")
tourney_data.dropna(subset=['KP_AdjEM_diff'], how='all',inplace=True)
train_data=tourney_data[tourney_data['Season']<2017].reset_index(drop=True)
test_data=tourney_data[tourney_data['Season']>2016].reset_index(drop=True)


col_names = ['Season','tourn_round','point_diff','ATeamID','ATeam','ASeed','Aregion','BTeamID','BTeam','BSeed','Bregion']
train_other_data=train_data.iloc[:,51:62]
train_output_data=train_data.iloc[:,64:65]
test_other_data=test_data.iloc[:,51:62]
test_output_data=test_data.iloc[:,64:65]
scaler = StandardScaler()

scaler.fit(train_input_data.iloc[:,:51])
train_input_data = pd.DataFrame(scaler.transform(train_data.iloc[:,:51]), index=train_data.iloc[:,:51].index, columns=train_data.iloc[:,:51].columns)
test_input_data = pd.DataFrame(scaler.transform(test_data.iloc[:,:51]), index=test_data.iloc[:,:51].index, columns=test_data.iloc[:,:51].columns)


test_full_data=test_input_data
test_full_data[col_names] = test_other_data 
test_full_data['AWon']=test_output_data
test_output_data['Season']=test_other_data['Season']

## Here we have a sample of some rows that will be used as inputs to predict the outcome of some March Madness games
##### Does not show all inputs, these inputs are an array of end of season stats for each team and then a comparison differential of said stats
###### Below we also have they testing output data which has the actual outcomes we will be trying to predict

In [128]:
#Show sample of DataFrame
test_input_data.head(5)

,A_KP_AdjEM,A_KP_SOS_Adj,ALuck,Asos,Ats_pct,...,BTeamID,BTeam,BSeed,Bregion,AWon
0,-2.051854,-2.137949,0.677966,-1.092878,0.854366,...,1112,arizona,2,X,0
1,0.593646,0.241447,0.631611,0.807216,1.093037,...,1388,saint-marys,7,X,1
2,-0.506846,0.710257,0.909741,-1.259079,-0.513233,...,1116,arkansas,8,Z,0
3,0.698238,1.348359,0.098530,-0.152671,-0.215393,...,1308,new-mexico-st,14,W,1
4,-0.527309,-0.063652,1.396467,-0.568473,0.182276,...,1124,baylor,3,W,0


In [129]:
test_output_data.head(5)



,AWon,Season
0,0,2017
1,1,2017
2,0,2017
3,1,2017
4,0,2017


## Loading Model Framework and Weights
##### This loads a file that has the saved framework and tuned model, it then loads the weights which will be used for predicting games

In [130]:
#Load Model and weights
json_file = open('model_3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
optimizer = keras.optimizers.Adagrad(lr=.01,epsilon=0,decay=0)
model.compile(optimizer= optimizer ,loss='binary_crossentropy',metrics=['acc'])
model.load_weights('model_demo.h5')

print("Loaded model weights from disk")

Loaded model weights from disk


## How we show performance?
##### The primary basis for measuring a model performance is measuring 'Log Loss' or basically a measure of accuracy that factors in confidence
##### Below we will show: Log Loss for random choice of winner/confidence, The Log Loss for the Top 5 models as part of the Kaggle NCAA competition for the 3 years we are testing
##### Link to Kaggle Leaderboard for 2019: https://www.kaggle.com/c/mens-machine-learning-competition-2019/leaderboard

In [131]:
def LogLoss(predictions, realizations):
    predictions_use = predictions.clip(0)
    realizations_use = realizations.clip(0)
    LogLoss = -np.mean( (realizations_use * np.log(predictions_use)) + 
                        (1 - realizations_use) * np.log(1 - predictions_use) )
    return LogLoss

In [132]:
#For reference this is the LogLoss of a model that predicts 50/50 shot of either team winning each time
#rather than favoring one team or another

bench_5050 = np.repeat(0.5, len(Y_test))
LogLoss(bench_5050, Y_test.AWon)

0.6931471805599452

In [133]:
#Log Loss for past Kaggle competition winners
#https://www.kaggle.com/c/mens-machine-learning-competition-2019/leaderboard
pastScores

,Year,Rank,LogLoss
0,2017,1,0.43857
1,2017,2,0.44981
2,2017,3,0.45373
3,2017,4,0.46100
4,2017,5,0.46107
5,2018,1,0.53194
6,2018,2,0.53693
7,2018,3,0.54013
8,2018,4,0.54967
9,2018,5,0.54987


## How does my model stack up?
##### Here is a readout of accuracy followed by Log Loss for the 3 years of Kaggle competition shown above
##### We then show a sample of outputs for the models outputted prediction/confidence

In [134]:
#LogLoss FN for accuracy weighted by confidence


#Accuracy on Test set
years=[2017,2018,2019]
for year in years:
    test_input_temp=test_input_data[test_input_data['Season']==year].reset_index(drop=True)
    test_output_temp=test_output_data[test_output_data['Season']==year].reset_index(drop=True)
    test_output_temp=test_output_temp[['AWon']]
    
    scores = model.evaluate(test_input_temp.iloc[:,0:51], test_output_temp, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    #Test LogLoss on holdout test set
    y_pred_keras = model.predict(test_input_temp.iloc[:,0:51]).ravel()
    y_pred_round=np.round(y_pred_keras,0)
    temp = np.array(y_pred_keras)[np.newaxis]
    np.set_printoptions(suppress=True)
    y_pred_tran=temp.T
    print("LL_Test ",year,": ",LogLoss(y_pred_tran, test_output_temp))


y_pred_keras = model.predict(test_input_data.iloc[:,0:51]).ravel()
y_pred_round=np.round(y_pred_keras,0)
testPerf=test_input_data.iloc[:,0:51]
testPerf['y_pred_confidence']=y_pred_keras

testPerf['y_pred_test']=y_pred_round
testPerf['y_act']=test_output_data.AWon
testPerf.sample(10)

acc: 82.09%
LL_Test  2017 :  AWon    0.350345
dtype: float64
acc: 83.58%
LL_Test  2018 :  AWon    0.380307
dtype: float64
acc: 88.06%
LL_Test  2019 :  AWon    0.254205
dtype: float64


,A_KP_AdjEM,A_KP_SOS_Adj,ALuck,Asos,Ats_pct,...,AdjustD_diff,AdjustO_diff,y_pred_confidence,y_pred_test,y_act
184,1.357624,0.994891,-0.341842,1.147564,0.576038,...,-1.583046,-0.090864,0.973169,1.0,1
101,0.694828,0.834900,0.770676,0.284007,0.131591,...,-0.556506,0.053690,0.972400,1.0,1
69,-0.044139,0.023785,0.399837,0.169765,2.076259,...,-0.403912,0.439168,0.171841,0.0,0
195,0.785777,1.190229,-0.249132,-0.112805,0.941725,...,0.913943,-0.259510,0.052858,0.0,0
14,1.587272,-0.528741,0.028997,3.456051,2.514824,...,-0.445529,0.270522,0.584068,1.0,1
146,0.010431,-0.820817,0.330304,1.170740,1.271183,...,0.497778,0.463260,0.042339,0.0,0
180,0.926750,0.903734,0.353482,0.740840,1.238068,...,0.109358,0.065737,0.035963,0.0,0
60,1.032479,0.931639,-1.431183,0.677585,-0.672497,...,-1.527557,0.860784,0.964737,1.0,1
104,0.034305,-0.130624,-1.037166,0.119956,1.288333,...,1.052665,1.330585,0.809706,1.0,1
192,1.833973,0.994891,0.724321,2.357155,1.093317,...,-0.695227,1.523324,0.972823,1.0,1


# So that's cool, but how do we do at ACTUALLY picking a bracket?
#### We start with the 2017 bracket and show each rounds matchups as my model picked them, followed by the calculated ESPN score

In [135]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from runTournament import runTournament

df=pd.read_csv("tourney_game_master.csv")
#2017 results
dfList, scoreDF=runTournament(df,2017,'model_3','model_demo')
for df in dfList:
    print('\n',df)

scoreDF


                  ATeam             BTeam  ATeamID  BTeamID
0       mount-st-marys         villanova     1291     1437
1        virginia-tech         wisconsin     1439     1458
2             virginia    unc-wilmington     1438     1423
3    east-tennessee-st           florida     1190     1196
4                  usc               smu     1425     1374
5               baylor     new-mexico-st     1124     1308
6            marquette    south-carolina     1266     1376
7                 troy              duke     1407     1181
8      south-dakota-st           gonzaga     1355     1211
9           vanderbilt      northwestern     1435     1321
10          notre-dame         princeton     1323     1343
11       west-virginia          bucknell     1452     1137
12              xavier          maryland     1462     1268
13  florida-gulf-coast        florida-st     1195     1199
14                 vcu       saint-marys     1433     1388
15        north-dakota           arizona     1315     

,round,score
0,32,31
1,16,22
2,8,28
3,4,16
4,2,16
5,1,32
6,Total,145


# 2017 could have been a fluke, how about 2018?
#### Same process as before

In [136]:
#2018 results
df=pd.read_csv("tourney_game_master.csv")
dfList, scoreDF=runTournament(df,2018,'model_3','model_demo')
for df in dfList:
    print('\n',df)

scoreDF


                ATeam                  BTeam  ATeamID  BTeamID
0            radford              villanova     1347     1437
1            alabama          virginia-tech     1104     1439
2      west-virginia              murray-st     1452     1293
3           marshall             wichita-st     1267     1455
4            florida         st-bonaventure     1196     1382
5   stephen-f-austin             texas-tech     1372     1403
6           arkansas                 butler     1116     1139
7             purdue       cal-st-fullerton     1345     1168
8             kansas                   penn     1242     1335
9         seton-hall      north-carolina-st     1371     1301
10           clemson          new-mexico-st     1155     1308
11            auburn  college-of-charleston     1120     1158
12          syracuse                    tcu     1393     1395
13          bucknell            michigan-st     1137     1277
14          oklahoma           rhode-island     1328     1348
15    

,round,score
0,32,30
1,16,22
2,8,12
3,4,16
4,2,16
5,1,32
6,Total,128


# We did ok considering a 16 seed upset a 1 seed in the 1st round 2018. Finally, this last year:

In [137]:
#2019 results
df=pd.read_csv("tourney_game_master.csv")
dfList, scoreDF=runTournament(df,2019,'model_3','model_demo')
for df in dfList:
    print('\n',df)

scoreDF



                 ATeam                BTeam  ATeamID  BTeamID
0                duke      north-dakota-st     1181     1295
1                 ucf                  vcu     1416     1433
2             liberty       mississippi-st     1251     1280
3         saint-louis        virginia-tech     1387     1439
4             belmont             maryland     1125     1268
5                yale                  lsu     1463     1261
6          louisville            minnesota     1257     1278
7         michigan-st              bradley     1277     1133
8             gonzaga  fairleigh-dickinson     1211     1192
9            syracuse               baylor     1393     1124
10          murray-st            marquette     1293     1266
11            vermont           florida-st     1436     1199
12            buffalo           arizona-st     1138     1113
13         texas-tech    northern-kentucky     1403     1297
14             nevada              florida     1305     1196
15           michigan 

,round,score
0,32,27
1,16,32
2,8,32
3,4,24
4,2,16
5,1,32
6,Total,163


# 2019 went pretty well, but how well?
#### Below is my models score compared to the average ESPN score and the score given that you just pick higher seeds
#### Also shown is a percentile score compared to all other users on ESPN
#### Here's a link to the 2019 leaderboard: http://fantasy.espn.com/tournament-challenge-bracket/2019/en/group?groupID=1041234
#### Another note Nate Silver's Fivethirtyeight would only get 72 in 2017, 114 in 2018, and 95 in 2019: https://projects.fivethirtyeight.com/2018-march-madness-predictions/
#### Another point you will notice my model actually picked each of the last 3 champions
#### So not only is the model superior on statistical metrics, but it is good at the element humans care about - winning your office pool

In [139]:
pastMadness

,Year,avg_score,higherSeed_score,myScore,percentile_myScore
0,2017,65.7,82,145,99.99
1,2018,57.0,81,128,98.80
2,2019,74.0,124,163,99.99


# Now could this make me a millionaire?
#### Well, if I worked for Berkshire in 2019, ya.
#### See this article: https://www.si.com/extra-mustard/2019/03/19/warren-buffett-ncaa-tournament-bracket-pool-million-dollars-year-sweet-16

# Finally, are bigger companies with lots of staff doing better?
#### Not quite, as far as I could find the only company publishing any kind of results is Adobe
#### There model achieve performance that garnered the 98th percentile, not quite 99.99...
#### See this page: https://www.adobe.com/analytics/hack-the-bracket.html?red=a